# 🔬 Advanced ARC Dataset Analysis for 60%+ Score

Comprehensive analysis to identify patterns and optimize solution components.

In [ ]:
import json
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter, defaultdict
from scipy import ndimage
from sklearn.cluster import KMeans
import pandas as pd
from typing import Dict, List, Tuple, Any
import warnings
warnings.filterwarnings('ignore')

# Set style for better plots
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print("🔬 Advanced ARC Dataset Analysis Initialized")

In [ ]:
# Load comprehensive ARC data
def load_all_arc_data():
    """Load all ARC datasets for comprehensive analysis"""
    data_path = '/kaggle/input/arc-prize-2025'  # Adjust path as needed
    
    datasets = {}
    
    # Load all available datasets
    files_to_load = [
        ('train_challenges', 'arc-agi_training_challenges.json'),
        ('train_solutions', 'arc-agi_training_solutions.json'),
        ('eval_challenges', 'arc-agi_evaluation_challenges.json'),
        ('eval_solutions', 'arc-agi_evaluation_solutions.json'),
        ('test_challenges', 'arc-agi_test_challenges.json')
    ]
    
    for key, filename in files_to_load:
        try:
            with open(f'{data_path}/{filename}', 'r') as f:
                datasets[key] = json.load(f)
            print(f"✅ Loaded {key}: {len(datasets[key])} items")
        except FileNotFoundError:
            print(f"⚠️  {filename} not found, skipping")
            datasets[key] = {}
    
    return datasets

# Load data
arc_datasets = load_all_arc_data()
print(f"\n📊 Total datasets loaded: {len([k for k, v in arc_datasets.items() if v])}")

In [ ]:
# Advanced transformation pattern analyzer
class TransformationAnalyzer:
    def __init__(self):
        self.patterns = defaultdict(int)
        self.pattern_examples = defaultdict(list)
        self.complexity_scores = {}
        
    def analyze_transformation(self, input_grid, output_grid, task_id, example_id):
        """Analyze a single input->output transformation"""
        inp = np.array(input_grid)
        out = np.array(output_grid)
        
        patterns_found = []
        
        # 1. Identity transformation
        if np.array_equal(inp, out):
            patterns_found.append('identity')
        
        # 2. Size changes
        if inp.shape != out.shape:
            if out.shape[0] > inp.shape[0] or out.shape[1] > inp.shape[1]:
                patterns_found.append('size_increase')
            else:
                patterns_found.append('size_decrease')
        
        # 3. Geometric transformations
        if inp.shape == out.shape:
            # Rotation checks
            for k in [1, 2, 3]:
                if np.array_equal(np.rot90(inp, k), out):
                    patterns_found.append(f'rotation_{k*90}')
                    break
            
            # Reflection checks
            if np.array_equal(np.fliplr(inp), out):
                patterns_found.append('reflection_horizontal')
            elif np.array_equal(np.flipud(inp), out):
                patterns_found.append('reflection_vertical')
        
        # 4. Color transformations
        inp_colors = set(np.unique(inp))
        out_colors = set(np.unique(out))
        
        if inp_colors != out_colors:
            if len(out_colors) > len(inp_colors):
                patterns_found.append('color_addition')
            elif len(out_colors) < len(inp_colors):
                patterns_found.append('color_reduction')
            else:
                patterns_found.append('color_substitution')
        
        # 5. Fill operations
        if np.sum(inp == 0) > np.sum(out == 0):
            patterns_found.append('fill_background')
        
        # 6. Pattern completion
        if self._is_pattern_completion(inp, out):
            patterns_found.append('pattern_completion')
        
        # 7. Object operations
        inp_objects = self._count_objects(inp)
        out_objects = self._count_objects(out)
        
        if out_objects > inp_objects:
            patterns_found.append('object_duplication')
        elif out_objects < inp_objects:
            patterns_found.append('object_removal')
        
        # 8. Symmetry operations
        if self._creates_symmetry(inp, out):
            patterns_found.append('symmetry_creation')
        
        # 9. Counting operations
        if self._involves_counting(inp, out):
            patterns_found.append('counting_operation')
        
        # 10. Line/connection operations
        if self._involves_connections(inp, out):
            patterns_found.append('connection_operation')
        
        # Record patterns
        for pattern in patterns_found:
            self.patterns[pattern] += 1
            if len(self.pattern_examples[pattern]) < 5:  # Store up to 5 examples
                self.pattern_examples[pattern].append({
                    'task_id': task_id,
                    'example_id': example_id,
                    'input': input_grid,
                    'output': output_grid
                })
        
        return patterns_found
    
    def _is_pattern_completion(self, inp, out):
        """Check if transformation completes a pattern"""
        # Simple heuristic: more non-zero cells in output
        return np.sum(out != 0) > np.sum(inp != 0) * 1.2
    
    def _count_objects(self, grid):
        """Count connected components (objects) in grid"""
        try:
            labeled, num_features = ndimage.label(grid != 0)
            return num_features
        except:
            return 0
    
    def _creates_symmetry(self, inp, out):
        """Check if output is more symmetric than input"""
        # Check horizontal symmetry
        out_h_sym = np.array_equal(out, np.fliplr(out))
        inp_h_sym = np.array_equal(inp, np.fliplr(inp))
        
        # Check vertical symmetry
        out_v_sym = np.array_equal(out, np.flipud(out))
        inp_v_sym = np.array_equal(inp, np.flipud(inp))
        
        return (out_h_sym and not inp_h_sym) or (out_v_sym and not inp_v_sym)
    
    def _involves_counting(self, inp, out):
        """Check if transformation involves counting"""
        # Heuristic: small output with values that could be counts
        if out.size <= 9 and np.max(out) <= 9:
            unique_inp = len(np.unique(inp[inp != 0]))
            return unique_inp > 0 and np.any(out == unique_inp)
        return False
    
    def _involves_connections(self, inp, out):
        """Check if transformation involves connecting elements"""
        # Heuristic: more connected components in output
        inp_edges = np.sum(np.diff(inp, axis=0) != 0) + np.sum(np.diff(inp, axis=1) != 0)
        out_edges = np.sum(np.diff(out, axis=0) != 0) + np.sum(np.diff(out, axis=1) != 0)
        return out_edges > inp_edges * 1.5

print("🔍 Transformation analyzer initialized")

In [ ]:
# Run comprehensive analysis
def run_comprehensive_analysis():
    """Run complete dataset analysis"""
    analyzer = TransformationAnalyzer()
    
    # Analyze training data
    train_challenges = arc_datasets.get('train_challenges', {})
    train_solutions = arc_datasets.get('train_solutions', {})
    
    print(f"🔬 Analyzing {len(train_challenges)} training tasks...")
    
    task_stats = {
        'grid_sizes': [],
        'color_usage': Counter(),
        'complexity_scores': [],
        'num_examples': [],
        'transformation_types': []
    }
    
    for task_id, task in train_challenges.items():
        if task_id not in train_solutions:
            continue
            
        task_stats['num_examples'].append(len(task['train']))
        
        # Analyze each training example
        for i, example in enumerate(task['train']):
            inp = np.array(example['input'])
            out = np.array(example['output'])
            
            # Grid size analysis
            task_stats['grid_sizes'].extend([inp.shape, out.shape])
            
            # Color usage analysis
            for color in np.unique(inp):
                task_stats['color_usage'][color] += np.sum(inp == color)
            for color in np.unique(out):
                task_stats['color_usage'][color] += np.sum(out == color)
            
            # Transformation analysis
            patterns = analyzer.analyze_transformation(inp.tolist(), out.tolist(), task_id, i)
            task_stats['transformation_types'].extend(patterns)
            
            # Complexity score (simple heuristic)
            complexity = len(np.unique(inp)) + len(np.unique(out)) + abs(inp.size - out.size)
            task_stats['complexity_scores'].append(complexity)
    
    return analyzer, task_stats

# Run analysis
print("🚀 Starting comprehensive analysis...")
transformation_analyzer, dataset_stats = run_comprehensive_analysis()
print("✅ Analysis complete!")

In [ ]:
# Visualize analysis results
def create_analysis_visualizations(analyzer, stats):
    """Create comprehensive visualizations of dataset analysis"""
    fig, axes = plt.subplots(3, 3, figsize=(20, 15))
    fig.suptitle('🔬 ARC Dataset Comprehensive Analysis', fontsize=16, fontweight='bold')
    
    # 1. Top transformation patterns
    top_patterns = dict(Counter(analyzer.patterns).most_common(15))
    axes[0,0].barh(list(top_patterns.keys()), list(top_patterns.values()))
    axes[0,0].set_title('Top 15 Transformation Patterns')
    axes[0,0].set_xlabel('Frequency')
    
    # 2. Grid size distribution
    size_strings = [f"{s[0]}x{s[1]}" for s in stats['grid_sizes']]
    size_counts = Counter(size_strings)
    top_sizes = dict(size_counts.most_common(10))
    axes[0,1].bar(range(len(top_sizes)), list(top_sizes.values()))
    axes[0,1].set_xticks(range(len(top_sizes)))
    axes[0,1].set_xticklabels(list(top_sizes.keys()), rotation=45)
    axes[0,1].set_title('Top 10 Grid Sizes')
    axes[0,1].set_ylabel('Frequency')
    
    # 3. Color usage distribution
    colors = list(range(10))
    color_counts = [stats['color_usage'].get(c, 0) for c in colors]
    bars = axes[0,2].bar(colors, color_counts)
    axes[0,2].set_title('Color Usage Distribution')
    axes[0,2].set_xlabel('Color')
    axes[0,2].set_ylabel('Usage Count')
    
    # Color the bars with actual colors
    color_map = ['black', 'blue', 'red', 'green', 'yellow', 'gray', 'magenta', 'orange', 'lightblue', 'brown']
    for bar, color in zip(bars, color_map):
        bar.set_color(color)
    
    # 4. Complexity distribution
    axes[1,0].hist(stats['complexity_scores'], bins=20, alpha=0.7, edgecolor='black')
    axes[1,0].set_title('Task Complexity Distribution')
    axes[1,0].set_xlabel('Complexity Score')
    axes[1,0].set_ylabel('Frequency')
    
    # 5. Number of training examples per task
    axes[1,1].hist(stats['num_examples'], bins=range(1, max(stats['num_examples'])+2), alpha=0.7, edgecolor='black')
    axes[1,1].set_title('Training Examples per Task')
    axes[1,1].set_xlabel('Number of Examples')
    axes[1,1].set_ylabel('Number of Tasks')
    
    # 6. Pattern co-occurrence heatmap
    pattern_cooccurrence = defaultdict(lambda: defaultdict(int))
    # This would require more complex analysis - simplified for now
    axes[1,2].text(0.5, 0.5, 'Pattern Co-occurrence\n(Advanced Analysis)', 
                   ha='center', va='center', transform=axes[1,2].transAxes, fontsize=12)
    axes[1,2].set_title('Pattern Relationships')
    
    # 7. Grid size vs complexity scatter
    grid_areas = [s[0] * s[1] for s in stats['grid_sizes'][:len(stats['complexity_scores'])]]
    axes[2,0].scatter(grid_areas, stats['complexity_scores'], alpha=0.6)
    axes[2,0].set_title('Grid Size vs Complexity')
    axes[2,0].set_xlabel('Grid Area (cells)')
    axes[2,0].set_ylabel('Complexity Score')
    
    # 8. Pattern frequency pie chart
    top_5_patterns = dict(Counter(analyzer.patterns).most_common(5))
    other_count = sum(analyzer.patterns.values()) - sum(top_5_patterns.values())
    if other_count > 0:
        top_5_patterns['Others'] = other_count
    
    axes[2,1].pie(top_5_patterns.values(), labels=top_5_patterns.keys(), autopct='%1.1f%%')
    axes[2,1].set_title('Top Pattern Distribution')
    
    # 9. Summary statistics
    summary_text = f"""
    📊 DATASET SUMMARY
    
    Total Patterns: {len(analyzer.patterns)}
    Most Common: {Counter(analyzer.patterns).most_common(1)[0][0]}
    Avg Complexity: {np.mean(stats['complexity_scores']):.1f}
    Most Used Color: {stats['color_usage'].most_common(1)[0][0]}
    Common Grid Size: {Counter(size_strings).most_common(1)[0][0]}
    Avg Examples/Task: {np.mean(stats['num_examples']):.1f}
    """
    
    axes[2,2].text(0.1, 0.9, summary_text, transform=axes[2,2].transAxes, 
                   fontsize=10, verticalalignment='top', fontfamily='monospace')
    axes[2,2].set_title('Summary Statistics')
    axes[2,2].axis('off')
    
    plt.tight_layout()
    plt.show()
    
    return top_patterns, top_sizes

# Create visualizations
print("📊 Creating analysis visualizations...")
top_patterns, top_sizes = create_analysis_visualizations(transformation_analyzer, dataset_stats)
print("✅ Visualizations complete!")

In [ ]:
# Generate actionable insights for solution improvement
def generate_improvement_insights(analyzer, stats, top_patterns):
    """Generate specific insights for improving the solution"""
    insights = {
        'missing_rules': [],
        'optimization_opportunities': [],
        'neural_network_targets': [],
        'ensemble_improvements': []
    }
    
    # Identify missing rules based on pattern frequency
    current_rules = {
        'identity', 'fill_background', 'reflection_horizontal', 'reflection_vertical',
        'rotation_90', 'rotation_180', 'rotation_270', 'color_substitution'
    }
    
    for pattern, count in top_patterns.items():
        if pattern not in current_rules and count > 10:  # Significant frequency
            insights['missing_rules'].append({
                'pattern': pattern,
                'frequency': count,
                'priority': 'HIGH' if count > 50 else 'MEDIUM',
                'examples': analyzer.pattern_examples.get(pattern, [])[:3]
            })
    
    # Optimization opportunities
    if stats['color_usage'][0] > sum(stats['color_usage'].values()) * 0.5:
        insights['optimization_opportunities'].append({
            'type': 'background_optimization',
            'description': 'Background (color 0) dominates - optimize background handling',
            'impact': 'HIGH'
        })
    
    # Neural network targets
    complex_patterns = [p for p, c in top_patterns.items() 
                       if 'completion' in p or 'connection' in p or 'counting' in p]
    
    if complex_patterns:
        insights['neural_network_targets'].extend([
            {
                'pattern': pattern,
                'reason': 'Complex pattern requiring learned representations',
                'architecture': 'CNN + Attention' if 'completion' in pattern else 'Transformer'
            } for pattern in complex_patterns[:5]
        ])
    
    # Ensemble improvements
    total_patterns = sum(top_patterns.values())
    coverage_estimate = sum(count for pattern, count in top_patterns.items() 
                           if pattern in current_rules) / total_patterns
    
    insights['ensemble_improvements'].append({
        'current_coverage': f"{coverage_estimate:.1%}",
        'improvement_potential': f"{(1-coverage_estimate):.1%}",
        'recommendation': 'Focus on top missing patterns for maximum impact'
    })
    
    return insights

# Generate insights
print("💡 Generating improvement insights...")
improvement_insights = generate_improvement_insights(transformation_analyzer, dataset_stats, top_patterns)

# Display insights
print("\n🎯 KEY INSIGHTS FOR 60%+ SCORE:")
print("="*50)

print("\n🔧 MISSING RULES (High Priority):")
for rule in improvement_insights['missing_rules'][:5]:
    print(f"  • {rule['pattern']}: {rule['frequency']} occurrences ({rule['priority']} priority)")

print("\n🚀 OPTIMIZATION OPPORTUNITIES:")
for opt in improvement_insights['optimization_opportunities']:
    print(f"  • {opt['type']}: {opt['description']} (Impact: {opt['impact']})")

print("\n🧠 NEURAL NETWORK TARGETS:")
for target in improvement_insights['neural_network_targets'][:3]:
    print(f"  • {target['pattern']}: {target['reason']} -> {target['architecture']}")

print("\n📈 ENSEMBLE IMPROVEMENTS:")
for imp in improvement_insights['ensemble_improvements']:
    print(f"  • Current Coverage: {imp['current_coverage']}")
    print(f"  • Improvement Potential: {imp['improvement_potential']}")
    print(f"  • Recommendation: {imp['recommendation']}")

print("\n✅ Analysis complete! Ready for solution enhancement.")